In [1]:
""" Importando las librerias """
from bs4 import BeautifulSoup #(bs4)Es una Biblioteca que nos da el html
from selenium import webdriver # contorlador del navegador
from selenium.common.exceptions import WebDriverException #Manejar excepciones del navegador
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import os 
from os import environ #Importa el diccionario completo con todas las variables de entorno como PATH, HOME, USERNAME, etc.
from os import pathsep #se utiliza para separar las diferentes rutas en la variable de entorno
from datetime import date #poder obtener la fecja de extracion
import time
import re
import pandas as pd #poder manipular los datos en excel
import openpyxl


In [2]:
""" Declaracion de la ubicacion y ejecucion del controlador """
chrome_driver_path = 'C:/Users/LEANM/OneDrive/Escritorio/Extractor ebay/chromedriver' #Declarando ruta del driver
os.environ["PATH"] += os.pathsep + chrome_driver_path # parchando la ruta (dejandola fija)
#crear el objeto del controlador que nos abrira googleDrive
url = webdriver.Chrome() #Abrir googleDrive

In [3]:
""" Informacion del sitio a extraer """
main_site = 'https://www.ebay.com/' #Pagina a la que se extareran los datos
url.get(main_site+"/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313&_nkw=ryzen&_sacat=0") #Personalisando lo que busca driver


In [4]:
""" Declaramos el dataframe vacio"""
data_frame_title =[]
data_frame_product_status =[]
data_frame_price =[]
data_frame_seller_score =[]
data_frame_price_before =[]
data_frame_link =[]


while True: #mientras page haya  informacion
    page = BeautifulSoup(url.page_source,'html.parser') # page nos almacena todo el html
    """ Recorrer la sopa (Buscar la informacion en concreto) """
    for content in page.find_all('li',class_='s-item'): #Recorro page en contenido para que contenido tenga toda la informacion de page(el parser)

        #Titulo
        title = content.find('div',class_='s-item__title') #Definiendo variable
        if title: #si esto es verdad
            data_frame_title.append(title.text) #imprimir mensaje
        else: data_frame_title.append('No se encontro titulo') # Imprimir no se encontro

        #Estado del producto
        product_status = content.find('span',class_='SECONDARY_INFO') #Definiendo variable
        if product_status:#si esto es verdad
            data_frame_product_status.append(product_status.text)#imprimir mensaje
        else: data_frame_product_status.append('No se encontro estado') # Imprimir no se encontro

        #Precio
        price = content.find('span',class_='s-item__price')#Definiendo variable
        if price:#si esto es verdad
            price = re.sub(r'\s+', '.', price.text.replace('COP $', '').replace('$20', 'None').split('.')[0]) #price = re.sub(r'\s+','.',(price.text.replace('.',',').replace('COP $','')).split(',')[0])
            print(price)
            data_frame_price.append(price)#imprimir mensaje
        else: data_frame_price.append('No se encontro precio') # Imprimir no se encontro

        #Puntuacion vendedor
        seller_score = content.find('span',class_='s-item__etrs-text')#Definiendo variable
        if seller_score: #si esto es verdad
            data_frame_seller_score.append(seller_score.text)#imprimir mensaje
        else: data_frame_seller_score.append('No se encontro Puntuacion_vendedor') # Imprimir no se encontro
        
        #Precio de antes
        price_before = content.find('span',class_='STRIKETHROUGH')#Definiendo variable
        if price_before: #si esto es verdad
            data_frame_price_before.append(price_before.text)#imprimir mensaje
        else: data_frame_price_before.append('No se encontro precio de antes') # Imprimir no se encontro
        
        #link
        link = content.find('a',class_='s-item__link')#Definiendo variable
        if link: #si esto es verdad
            data_frame_link.append(link['href'])#imprimir mensaje
        else: data_frame_link.append('No se encontro enlace') # Imprimir no se encontro

    next_site = url.find_elements(By.CSS_SELECTOR, '.pagination__next')
    if len(next_site) == 0:
        break

    next_site[0].click()
    time.sleep(3)


None
551.020
1.191.836
832.653
5.300
191.836
566.530
605.057
1.506.122
526.530
791.836
2.204.040
5.300
1.791.836
1.510.000
693.755
1.424.489
1.302.908
5.300
2.081.591
530.612
1.693.836
42.857
122.448
1.206.847
1.306.122
776.347
297.918
326.489
187.714
5.300
693.877
428.359
302.040
252.382
322.448
571.428
1.761.627
321.269
771.428
1.512.204
1.751.020
485.714
408.163
1.101.836
481.591
1.224.448
644.897
1.686.664
535.502
346.938
265.306
163.224
40.775
244.897
244.897
975.510
183.673
1.972.944
1.653.061
803.173
None
481.591
506.122
1.972.944
265.306
550.816
192.761
383.673
81.591
191.836
118.367
428.571
224.489
321.269
1.628.571
481.850
1.156.569
232.653
893.877
208.122
138.734
40.105
326.489
677.551
476.549
514.244
473.711
506.081
1.183.673
1.512.204
1.416.285
803.173
669.311
1.224.489
204.081
477.551
81.632
873.469
653.061
1.653.061
803.119
163.224
281.632
469.346
693.877
81.632
200.000
244.857
46.938
1.751.020
1.918.326
1.061.224
118.367
639.755
5.300
142.857
1.632.653
232.653
314.285
1

KeyboardInterrupt: 

In [5]:
""" Declaramos el dataframe y le pasamos un diccionario como argumento """
together_data_frame_list = pd.DataFrame({
    'TITULO':data_frame_title,
    'ESTADO DEL PRODUCTO':data_frame_product_status,
    'PRECIO':data_frame_price,
    'PUNTAJE DEL VENDEDOR':data_frame_seller_score,
    'PRECIO ANTES':data_frame_price_before,
    'LINK':data_frame_link
})

In [6]:
""" Creando el csv (Exportar a excel)"""
# Ruta del archivo de exportación
file_path = r'C:\Users\LEANM\OneDrive\Escritorio\tutorial scraping ebay\app\data\dataset.xlsx'

# Exportar el DataFrame a un archivo de Excel
together_data_frame_list.to_excel(r'C:\Users\LEANM\OneDrive\Escritorio\tutorial scraping ebay\app\data\dataset.xlsx', index=False, header=True, sheet_name='Datos')

# Verificar si los datos se han actualizado correctamente
if os.path.isfile(file_path):
    # Cargar el DataFrame exportado desde el archivo
    data_frame_exportado = pd.read_excel(file_path, sheet_name='Datos')
    
    # Comparar los datos originales con los exportados
    if together_data_frame_list.equals(data_frame_exportado):
        print('¡Se exportaron los datos con éxito!\nLos datos exportados son iguales a los datos originales.')
    else:
        print('¡Se exportaron los datos con éxito!\nLos datos exportados son diferentes a los datos originales.')

else:
    print('Hubo un error con los datos elimine el excel para poder exportar.')

¡Se exportaron los datos con éxito!
Los datos exportados son diferentes a los datos originales.
